In [16]:
import csv
import json
import os
import requests
import pandas as pd
from dotenv import load_dotenv

Loading json file containing hotel data

In [17]:
json_file = "Singapore.json"  # to edit
filepath = "/Users/pritish/projects/Hotels-Decision-Support-and-Sentimental-Analysis" #to edit

with open(os.path.join(filepath,json_file)) as file:
    data = json.load(file)

Requesting geocode data through geocodeapi using hotel addresses.

In [18]:
load_dotenv()
API_KEY = os.getenv('GEOCODE_API_KEY')
url = 'https://app.geocodeapi.io/api/v1/search'

for i in data.keys():
    address = data[i]['address']
    response = requests.get(url, params = {'apikey': API_KEY,'text': address}).json()
    # accessing coordinates
    features = response['features']
    geometry = features[0]['geometry']
    coord = geometry['coordinates']
    # Add geocode data to hotel data
    data[i]['geocode'] = coord
    print(i)

The Fullerton Hotel Singapore
Shangri-La Hotel, Singapore
Pan Pacific Singapore Hotel
Orchard Hotel Singapore
PARKROYAL COLLECTION Marina Bay, Singapore Hotel
Marina Bay Sands Hotel, Singapore
Goodwood Park Hotel, Singapore
Park Hotel Alexandra, Singapore
Park Hotel Clarke Quay, Singapore
PARKROYAL COLLECTION Pickering, Singapore Hotel
Carlton Hotel Singapore
Capri by Fraser Changi City Hotel, Singapore
YOTEL Singapore Hotel
Ascott Orchard Singapore Hotel
Mandarin Orchard Singapore Hotel
Lloyd's Inn Singapore Hotel
Grand Park Orchard Hotel, Singapore
Hotel NuVe Heritage, Singapore
Hotel G Singapore
YOTELAIR Singapore Changi Airport Hotel


Adding co-ordinates to JSON data files

In [19]:
with open(os.path.join(filepath,json_file), 'w') as outfile:
            json.dump(data, outfile, indent=4)

Extracting hotel names and geocodes into a Data Frame

In [20]:
to_add = []
columns = ['Hotel_Name','Longitude','Latitude']
for i in data.keys():
    values = [i, data[i]['geocode'][0], data[i]['geocode'][1]]
    zipped_dict = dict(zip(columns, values))
    to_add.append(zipped_dict)
# Making pandas dataframe
df = pd.DataFrame(to_add)
df.head()

,Hotel_Name,Longitude,Latitude
0,The Fullerton Hotel Singapore,103.853122,1.286165
1,"Shangri-La Hotel, Singapore",103.833828,1.316382
2,Pan Pacific Singapore Hotel,103.833828,1.316382
3,Orchard Hotel Singapore,103.828164,1.307669
4,"PARKROYAL COLLECTION Marina Bay, Singapore Hotel",103.833828,1.316382


Write geocode pertinent data to a csv file


In [21]:
output_filename = json_file[:-5]
with open(os.path.join(filepath,f'{output_filename}_hotelsGeocode.csv'), 'w', newline='') as file:
    df.to_csv(file)